# Task 01 : Data Cleaning & Modeling
- Identify which datasets will be required to answer the client’s business question
- Clean the datasets and merge them to prepare the data for analysis
- Determine the answer to the client’s business question

## The main Question to answer is : top 5 categories with the largest popularity.
**to figure out popularity, we’ll have to add up which content categories have the largest score**

**Selected datasets** : Reaction , Content , Reaction Types

## Task 1.1

**Clean the data by:**

- removing rows that have values which are missing,
- changing the data type of some values within a column, and
- removing columns which are not relevant to this task.
Think about how each column might be relevant to the business question you’re investigating. If you can’t think of why a column may be useful, it may not be worth including it.

**1_Content dataset**

In [105]:
# Load dataset without the index column
import pandas as pd
df_content = pd.read_csv('Content.csv', index_col=0)
df_content.head()

,Content ID,User ID,Type,Category,URL
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,photo,Studying,https://socialbuzz.cdn.com/content/storage/975...
1,9f737e0a-3cdd-4d29-9d24-753f4e3be810,beb1f34e-7870-46d6-9fc7-2e12eb83ce43,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/9f7...
2,230c4e4d-70c3-461d-b42c-ec09396efb3f,a5c65404-5894-4b87-82f2-d787cbee86b4,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/230...
3,356fff80-da4d-4785-9f43-bc1261031dc6,9fb4ce88-fac1-406c-8544-1a899cee7aaf,photo,technology,https://socialbuzz.cdn.com/content/storage/356...
4,01ab84dd-6364-4236-abbb-3f237db77180,e206e31b-5f85-4964-b6ea-d7ee5324def1,video,food,https://socialbuzz.cdn.com/content/storage/01a...


In [106]:
#Removing rows with missing values 
df_content = df_content.dropna()

In [107]:
# Remove "URL" column
df_content = df_content.drop(columns=['URL'])
# Remove "Type" column
df_content = df_content.drop(columns=['Type'])


In [108]:
# Checking the types of the columns
print(df_content.dtypes)

# Set   Category column to String

df_content['Category'] = df_content['Category'].astype(str)



Content ID    object
User ID       object
Category      object
dtype: object


**2_Reactions dataset**

In [109]:
#Load dataset
df_reactions = pd.read_csv("Reactions.csv", index_col=0)
df_reactions.head()

,Content ID,User ID,Type,Datetime
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,NaN,NaN,2021-04-22 15:17:15
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust,2020-11-07 09:43:50
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,92b87fa5-f271-43e0-af66-84fac21052e6,dislike,2021-06-17 12:22:51
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared,2021-04-18 05:13:58
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust,2021-01-06 19:13:01


In [110]:
#Removing rows with missing values 
df_reactions = df_reactions.dropna()

In [111]:
#Removing Datetime column 
df_reactions = df_reactions.drop(columns=["Datetime"])

**3_Reaction Types dataset**

In [112]:
#Load dataset
df_reaction_types = pd.read_csv("ReactionTypes.csv", index_col=0)
df_reaction_types.head()

,Type,Sentiment,Score
0,heart,positive,60
1,want,positive,70
2,disgust,negative,0
3,hate,negative,5
4,interested,positive,30


In [113]:
#Removing rows with missing values 
df_reaction_types = df_reaction_types.dropna()

In [114]:
#Removing "Sentiment" column
df_reaction_types = df_reaction_types.drop(columns=['Sentiment'])

**Create a final data set by merging the three datasets together**

In [115]:
# Merging Reactions and Content datasets on User ID and Content ID

df_1 = df_reactions.merge(df_content, on=["User ID", "Content ID"])
df_1.head()

,Content ID,User ID,Type,Category
0,36d36f19-7a10-4d7d-a3ab-a3f2cbbfcf4a,69532ce2-1ffe-4187-8042-0230d115b24e,worried,food
1,f1038e6c-5292-4e20-a866-675e1637d6b5,13f0db8a-152a-496f-a6e8-1ed6a90b8788,cherish,soccer
2,ff883828-a610-492d-8635-8a777eaad25f,ded166ee-173b-4a56-bc6d-464c35709670,peeking,education
3,a3e93c2d-7fb4-4881-994d-ee20ad564924,317c70d5-fe65-4663-afb1-150108e0e55c,adore,public speaking
4,b4c71928-99f0-4f1e-bdff-3d99c16113ac,bf721d00-4fa1-44b1-834f-4730cd51aefa,scared,travel


In [116]:
# Merging df_1 with reactions type dataset on "Type"
df_final = df_1.merge(df_reaction_types, on="Type")
df_final.head()

,Content ID,User ID,Type,Category,Score
0,36d36f19-7a10-4d7d-a3ab-a3f2cbbfcf4a,69532ce2-1ffe-4187-8042-0230d115b24e,worried,food,12
1,f1038e6c-5292-4e20-a866-675e1637d6b5,13f0db8a-152a-496f-a6e8-1ed6a90b8788,cherish,soccer,70
2,ff883828-a610-492d-8635-8a777eaad25f,ded166ee-173b-4a56-bc6d-464c35709670,peeking,education,35
3,a3e93c2d-7fb4-4881-994d-ee20ad564924,317c70d5-fe65-4663-afb1-150108e0e55c,adore,public speaking,72
4,b4c71928-99f0-4f1e-bdff-3d99c16113ac,bf721d00-4fa1-44b1-834f-4730cd51aefa,scared,travel,15


In [117]:
#remove UserID column 
df_final = df_final.drop(columns="User ID")

In [118]:
df_final.head()

,Content ID,Type,Category,Score
0,36d36f19-7a10-4d7d-a3ab-a3f2cbbfcf4a,worried,food,12
1,f1038e6c-5292-4e20-a866-675e1637d6b5,cherish,soccer,70
2,ff883828-a610-492d-8635-8a777eaad25f,peeking,education,35
3,a3e93c2d-7fb4-4881-994d-ee20ad564924,adore,public speaking,72
4,b4c71928-99f0-4f1e-bdff-3d99c16113ac,scared,travel,15


In [119]:
# Save the df_final dataset to a CSV file named clean_df.csv
df_final.to_csv("clean_df.csv", index=False)

In [120]:
# Calculate the total score for each category
category_popularity = df_final.groupby("Category")["Score"].sum().reset_index()

# Sort the categories by total score in descending order
category_popularity = category_popularity.sort_values(by="Score", ascending=False)

# Get the top 5 categories with the largest popularity
top_5_categories = category_popularity.head(5)
top_5_categories

,Category,Score
6,public speaking,232
7,science,227
2,culture,150
0,animals,145
3,education,140


In [121]:
import pandas as pd

# Highlight the top 5 categories in the df_final dataframe
highlighted_df = df_final[df_final["Category"].isin(top_5_categories["Category"])]

# Save the highlighted data to an Excel file
highlighted_df.to_excel("highlighted_top_5_categories.xlsx", index=False)

## The top 5 categories are : 
- public_speaking 
- science
- culture
- animals
- education